In [1]:
#! conda install matplotlib torch pandas numpy scikit-learn torchvision tensorflow tensorboard tqdm seaborn biopython xgboost 
#! pip install jupyter-server-proxy viennaRNA torchsummary 

In [2]:
import datetime
import itertools
from typing import Literal, Optional

import Bio.Seq
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary
from torchviz import make_dot
import tqdm
from xgboost import XGBRegressor, XGBClassifier
import ViennaRNA

import data_loading
import models
import train

2025-08-12 18:22:47.717065: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 18:22:48.790965: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [4]:
df = data_loading.build_dataset('frameshift_data_natcomm.csv') #supplementary data 2 file of paper
df.head()

,gfp_minus_rate,gfp_plus_rate,sequence,variant_id,prf_class,gfp_minus,gfp_plus
1506,0.86,NaN,GATAGGTACCTGGCTGTCGTCCATGCTGTGTTTGCAAAAAAAGCCA...,37296,human - ccr5,False,False
1507,58.57,NaN,GATAGGTACCTGGCTGTCGTCCATGCTGTGTTTGCAAAAAACGCCA...,37297,human - ccr5,True,False
1508,NaN,71.48,GATAGGTACCTGGCTGTCGTCCATGCTGTGTTTGCAAAAAAGGCCA...,37298,human - ccr5,False,True
1509,NaN,1.30,GATAGGTACCTGGCTGTCGTCCATGCTGTGTTTGCAAAAAATGCCA...,37299,human - ccr5,False,False
1511,NaN,47.30,GATAGGTACCTGGCTGTCGTCCATGCTGTGTTTGCAAACCCCGCCA...,37301,human - ccr5,False,True


In [5]:
df['prf_class'].value_counts()

prf_class
HIV HXB2                             614
PLRV luteovirus                      612
human - OAZ1                         596
SARS coronavirus                     583
herpes simplex - thymidine kinase    582
human - PEG10                        575
simian srv1 retrovirus               562
human - HERV-K10                     525
Rous sarcoma virus                   511
influenza a virus                    456
SIVmac239                            450
synthetic                            450
west nile virus                      440
PRRSV - nsp2TF                       422
HCV - F protein                      378
human T-lymphotropic virus           369
human - ccr5                         276
Name: count, dtype: int64

In [6]:
PAPER_CAT = [
    'SARS coronavirus',
    'SIVmac239',
    'simian srv1 retrovirus',
    'PLRV luteovirus',
    'human - HERV-K10',
    'human T-lymphotropic virus',
    'HIV HXB2',
    'west nile virus',
    'mix'
]

In [7]:
baselines = {
    'mix': {
        'ROC-AUC/val': .89,
        'ROC-AUC/train': .89,
        'PR-AUC/val': .8,
        'PR-AUC/train': .8,
        'Pearson/val': .54,
        'Pearson/train': .54,
    },
    'SARS coronavirus': {
        'ROC-AUC/val': .63,
        'ROC-AUC/train': .63, 
        'Pearson/val': .24,
        'Pearson/train': .24,
    },
    'SIVmac239': {
        'ROC-AUC/val': .74,
        'ROC-AUC/train': .74,  
        'Pearson/val': .45,
        'Pearson/train': .45,
    },
    'simian srv1 retrovirus': {
        'ROC-AUC/val': .94,
        'ROC-AUC/train': .94,    
        'Pearson/val': .61,
        'Pearson/train': .61,
    },
    'PLRV luteovirus': {
        'ROC-AUC/val': .66,
        'ROC-AUC/train': .66,    
        'Pearson/val': .19,
        'Pearson/train': .19,
    },
    'human - HERV-K10': {
        'ROC-AUC/val': .85,
        'ROC-AUC/train': .85,   
        'Pearson/val': .63,
        'Pearson/train': .63,
    },
    'human T-lymphotropic virus': {
        'ROC-AUC/val': .88,
        'ROC-AUC/train': .88,   
        'Pearson/val': .39,
        'Pearson/train': .39,
    },
    'HIV HXB2': {
        'ROC-AUC/val': .89,
        'ROC-AUC/train': .89,      
        'Pearson/val': .69,
        'Pearson/train': .69,
    },
    'west nile virus': {
        'ROC-AUC/val': .47,
        'ROC-AUC/train': .47,   
        'Pearson/val': .61,
        'Pearson/train': .61,
    },
    
}

In [8]:
def secondary_features(sequence: str):
    _MAPPING = {'.': 0, '(': 1, ')': 1} 
    folded, _ = ViennaRNA.fold(sequence)
    encoded = [_MAPPING[pos] for pos in folded]
    return encoded

In [9]:
def aa_polarity_features(sequence: str):
    _AA_POLARITY = {
        'R':'charged',
        'H':'charged',
        'K':'charged',
        'D':'charged',
        'E':'charged',
        'S':'polar',
        'T':'polar',
        'N':'polar',
        'Q':'polar',
        'C':'polar',
        'Y':'polar',
        'G':'unpolar',
        'P':'unpolar',
        'A':'unpolar',
        'V':'unpolar',
        'I':'unpolar',
        'L':'unpolar',
        'M':'unpolar',
        'F':'unpolar',
        'W':'unpolar',
        '*':'stop'
    }
    translated = Bio.Seq.Seq(df.iloc[0].sequence).translate()
    polarity = [_AA_POLARITY[aa] for aa in translated]
    return polarity

In [10]:
tai_availability = pd.read_csv('tAI_index_human_nar-02315.txt', sep='\t').rename(columns={'w ': 'w'})
tai_availability.loc[:, 'w'] = tai_availability.loc[:, 'w'].map(lambda x: float(x))
tai_dict = dict(zip(tai_availability['Codon'], tai_availability['w']))
#tai_dict

In [11]:
def mikl_tai(df, tai_file):
    tai_availability = pd.read_csv(tai_file, sep='\t').rename(columns={'w ': 'w'})
    tai_availability.loc[:, 'w'] = tai_availability.loc[:, 'w'].map(lambda x: float(x))
    tai_dict = dict(zip(tai_availability['Codon'], tai_availability['w']))

    def _gettaimap(vrsq):
        tai=[]
        for i in range(len(vrsq)//3-1):
            tai.append(tai_dict[vrsq[0+3*i:3+3*i]])
        return pd.Series(tai)
    
    def _gettaidiffmap(vrsq):
        tai=[]
        for i in range(len(vrsq)//3-1):
            tai.append(tai_dict[vrsq[2+3*i:5+3*i]]-tai_dict[vrsq[3+3*i:6+3*i]])
        return pd.Series(tai)

    tai0=df.sequence.apply(lambda x: _gettaimap(x.upper()))
    taim1=df.sequence.apply(lambda x: _gettaimap(x[2:].upper()))
    taip1=df.sequence.apply(lambda x: _gettaimap(x[1:].upper()))
    taidiff=df.sequence.apply(lambda x: _gettaidiffmap(x.upper()))

    tai=tai0.join(taim1, lsuffix='_frame0', rsuffix = '_framem1')
    tai=tai.join(taip1, rsuffix = '_framep1')
    tai=tai.join(taidiff, rsuffix = '_diffframem1to0')

    return tai

def mikl_charged(df):
    aacategory={
        'R':'charged',
        'H':'charged',
        'K':'charged',
        'D':'charged',
        'E':'charged',
        'S':'polar',
        'T':'polar',
        'N':'polar',
        'Q':'polar',
        'C':'polar',
        'Y':'polar',
        'G':'unpolar',
        'P':'unpolar',
        'A':'unpolar',
        'V':'unpolar',
        'I':'unpolar',
        'L':'unpolar',
        'M':'unpolar',
        'F':'unpolar',
        'W':'unpolar',
        '*':'stop'
        }
    
    def getaaclassmap(vrsq):
        aa=[]
        for i in range(len(vrsq)//3-1):
            aa.append(aacategory[Bio.Seq.Seq(vrsq[0+3*i:3+3*i]).translate()])
        return pd.Series(aa)
    
    aa0=df.sequence.apply(lambda x: getaaclassmap(x.upper()))
    aam1=df.sequence.apply(lambda x: getaaclassmap(x[2:].upper()))
    aap1=df.sequence.apply(lambda x: getaaclassmap(x[1:].upper()))
    
    aa0enc=pd.DataFrame(index=aa0.index)
    for col in aa0.columns:
        labelenc=LabelEncoder()
        feature0=labelenc.fit_transform(aa0[col])
        onehotenc=OneHotEncoder(sparse_output=False)
        aa0enc=aa0enc.join(pd.DataFrame(onehotenc.fit_transform(feature0.reshape(feature0.shape[0],1)), index=aa0.index, columns=labelenc.classes_), rsuffix=col)
    
    aam1enc=pd.DataFrame(index=aam1.index)
    for col in aam1.columns:
        labelenc=LabelEncoder()
        feature0=labelenc.fit_transform(aam1[col])
        onehotenc=OneHotEncoder(sparse_output=False)
        aam1enc=aam1enc.join(pd.DataFrame(onehotenc.fit_transform(feature0.reshape(feature0.shape[0],1)), index=aam1.index, columns=labelenc.classes_), rsuffix=col)
    
    aap1enc=pd.DataFrame(index=aap1.index)
    for col in aap1.columns:
        labelenc=LabelEncoder()
        feature0=labelenc.fit_transform(aap1[col])
        onehotenc=OneHotEncoder(sparse_output=False)
        aap1enc=aap1enc.join(pd.DataFrame(onehotenc.fit_transform(feature0.reshape(feature0.shape[0],1)), index=aap1.index, columns=labelenc.classes_), rsuffix=col)
    
    aa=aa0enc.join(aam1enc, lsuffix='_frame0', rsuffix = '_framem1')
    aa=aa.join(aap1enc, rsuffix = '_framep1')
    return aa

def mikl_fold(df):
    sec=df.sequence.apply(lambda x: pd.Series(list(ViennaRNA.fold(x)[0])))
    sec=sec.replace({'.':0,'(':1,')':1})
    return sec

In [ ]:
%%time
aa = mikl_charged(df)
sec = mikl_fold(df)
tai = mikl_tai(df, 'tAI_index_human_nar-02315.txt')

In [ ]:
feat=tai.join(aa)
#feat=feat.join(fsdesforml[[x for x in fsdesforml.columns if 'dg' in x]])
feat=feat.join(sec)
feat=feat.join(df)
#feat=feat.join(slipenc)
seccols=list(np.arange(42,162))
taicols=[str(x)+'_frame0' for x in np.arange(12,26)]+[str(x)+'_framem1' for x in np.arange(12,26)]+[str(x)+'_diffframem1to0' for x in np.arange(12,26)]
aacols=['charged'+str(x) for x in np.arange(10, 14)]+['polar'+str(x) for x in np.arange(10, 14)]+['unpolar'+str(x) for x in np.arange(10, 14)]
all_cols = seccols+taicols+aacols

In [ ]:
dataset_splits=list(itertools.product(PAPER_CAT, PAPER_CAT))

mikl_perfs = []

for x, y in dataset_splits:
    if x == 'mix' or y == 'mix':
        continue
    if x == y:
        train_data, test_data = train_test_split(feat[feat['prf_class'] == x], test_size=.2, random_state=42)
    else:
        train_data = feat[feat['prf_class'] == x]
        test_data = feat[feat['prf_class'] == y]
    model = XGBClassifier()
    model.fit(train_data[all_cols], train_data['gfp_minus'])
    pred = model.predict_proba(test_data[all_cols])
    roc = roc_auc_score(test_data['gfp_minus'], pred[:,1])
    mikl_perfs.append({
        'train_data': x,
        'test_data': y,
        'roc_auc': roc
    })
mikl_df = pd.DataFrame.from_records(mikl_perfs)
tmp = mikl_df.pivot_table(columns="test_data", index="train_data", values='roc_auc', aggfunc="mean")
tmp = tmp.reindex(index=PAPER_CAT, columns=PAPER_CAT)
sns.heatmap(tmp, annot=True, fmt=".3f", linewidths=0.5, vmin=0, vmax=0.82, cmap=sns.cubehelix_palette(as_cmap=True))
plt.show()

In [ ]:
mikl_perfs = []

ablation = {
    'all': all_cols,
    'no tRNA availability': seccols+aacols,
    'no RNA secondary structure': taicols+aacols,
    'no AA charge': seccols+taicols,
    'only AA charge': aacols,
    'only tRNA availability': taicols,
    'only RNA secondary structure': seccols
}

for method in ablation:
    for x in PAPER_CAT:
        if x == 'mix':
            train_data, test_data = train_test_split(feat, test_size=.2, random_state=42)
        else:
            train_data = feat[feat['prf_class'] != x]
            test_data = feat[feat['prf_class'] == x]
        model = XGBClassifier()
        model.fit(train_data[ablation[method]], train_data['gfp_minus'])
        pred = model.predict_proba(test_data[ablation[method]])
        mikl_perfs.append({
            'train_data': 'leave_one',
            'test_data': x,
            'roc_auc': roc_auc_score(test_data['gfp_minus'], pred[:,1]),
            'pr_auc': average_precision_score(test_data['gfp_minus'], pred[:,1]),
            'model': f'Mikl {method}'
        })

mikl_df = pd.DataFrame.from_records(mikl_perfs)

In [ ]:
mikl_perfs = []

ablation = {
    'all': all_cols,
    'no tRNA availability': seccols+aacols,
    'no RNA secondary structure': taicols+aacols,
    'no AA charge': seccols+taicols,
    'only AA charge': aacols,
    'only tRNA availability': taicols,
    'only RNA secondary structure': seccols
}

for i in range(10):
    for method in ablation:
        for x in PAPER_CAT:
            if x == 'mix':
                train_data, test_data = train_test_split(feat, test_size=.2, random_state=42)
            else:
                train_data = feat[feat['prf_class'] != x]
                test_data = feat[feat['prf_class'] == x]
            model = XGBClassifier(seed = i)
            model.fit(train_data[ablation[method]], train_data['gfp_minus'])
            pred = model.predict_proba(test_data[ablation[method]])
            mikl_perfs.append({
                'train_data': 'leave_one',
                'test_data': x,
                'roc_auc': roc_auc_score(test_data['gfp_minus'], pred[:,1]),
                'pr_auc': average_precision_score(test_data['gfp_minus'], pred[:,1]),
                'model': f'Mikl {method}',
                'iter': i
            })

mikl_df = pd.DataFrame.from_records(mikl_perfs)

In [ ]:
ax = sns.barplot(mikl_df, x='roc_auc', y='test_data', hue='model')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xmin=0.4)
plt.show()

ax = sns.barplot(mikl_df, x='pr_auc', y='test_data', hue='model')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xmin=0.4)
plt.show()

In [ ]:
# Instantiate the model
model = models.RNACNN_pete()

input_shape = (5, 162)

# Generate summary
summary(model, input_shape)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cnn_df = train.main_loop(
    model_fns=[
        lambda: models.RNACNN_pete()
        #lambda: models.RNACNN_pete()
        #lambda: models.RNACNN(),
        #lambda: models.RNACNN()
        #lambda: models.RNACNN_FC(),
        #lambda: models.RNACNN_GN(),
        #lambda: models.RNACNN_FC_GN(),
     ],
    data=df,
    device=device,
    base_name='cnn',
    lrs=[1e-3,],
    #lrs=[1e-2, 1e-3, 1e-4, 1e-5,],
    wds=[0.01],
    #wds=[0.01, 0.05, 0.1, 0.2],
    schedulers = ['CyclicLR',],
    #schedulers = ['StepLR', 'ExponentialLR', 'CosineAnnealingLR', 'CyclicLR'],
    dataset_splits=PAPER_CAT,
    #baselines=baselines,
    epochs=200,
    loss_mixin=.999,
    data_augment=False
)
cnn_df = cnn_df.rename(columns={'mix': 'test_data'})

In [ ]:
import pandas as pd
from typing import Mapping

def load_trna_mapping(file_path: str) -> Mapping[str, float]:
    df = pd.read_csv(file_path, sep='\t')
    return dict(zip(df['Codon'], df['w']))

trna_av = data_loading.load_trna_mapping("tAI_index_human_nar-02315.txt")
print(trna_av["ATG"])  # Should print 0.611247 (w value for AUG)
print(trna_av["TTT"])  # Should print 0.216381

In [ ]:
import torch
print(f"Device: {device}")
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
print(f"GPU name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
cnn_df

In [ ]:
random_baseline = []
for x in PAPER_CAT:
    if x == 'mix':
        train_data, test_data = train_test_split(feat, test_size=.2, random_state=42)
    else:
        train_data = feat[feat['prf_class'] != x]
        test_data = feat[feat['prf_class'] == x]
    random_baseline.append({
        'test_data': x,
        'roc_auc': .5,
        'pr_auc': test_data['gfp_minus'].mean(),
        'model': f'Random baseline'
    })

total = pd.concat([cnn_df, mikl_df, pd.DataFrame.from_records(random_baseline)])


tmp = total.pivot_table(index="test_data", columns="model", values='roc_auc', aggfunc="mean")
tmp = tmp.reindex(columns=sorted(total['model'].unique()), index=PAPER_CAT)
sns.heatmap(tmp, annot=True, fmt=".3f", linewidths=0.5, cmap=sns.cubehelix_palette(as_cmap=True))
plt.show()

In [ ]:
mikl_df

In [ ]:
ablation = {
    'all': all_cols,
    'no tRNA availability': seccols+aacols,
    'no RNA secondary structure': taicols+aacols,
    'no AA charge': seccols+taicols,
    'only AA charge': aacols,
    'only tRNA availability': taicols,
    'only RNA secondary structure': seccols
}

MIKL_ORDER = [
    'Mikl all',
    'Mikl only tRNA availability',
    'Mikl no tRNA availability'
]

ax = sns.barplot(mikl_df, x='roc_auc', y='test_data', hue='model', hue_order=MIKL_ORDER)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xmin=0.4)
plt.show()

ax = sns.barplot(mikl_df, x='pr_auc', y='test_data', hue='model', hue_order=MIKL_ORDER)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xmin=0.4)
plt.show()

In [ ]:
ax = sns.barplot(total, x='roc_auc', y='test_data', hue='model', hue_order=['RNACNN_pete', 'Mikl all', 'Random baseline'])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xmin=0.4)
plt.show()

In [ ]:
ax = sns.barplot(total, x='roc_auc', y='test_data', hue='model', hue_order=['RNACNN_pete', 'Mikl all', 'Random baseline'])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.xlim(xmin=0.4)
plt.show()

In [ ]:
ax = sns.barplot(total, x='pr_auc', y='test_data', hue='model', hue_order=['RNACNN_pete', 'Mikl all', 'Random baseline'])
#plt.xlim(xmin=0.5)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
ax = sns.barplot(total, x='pr_auc', y='test_data', hue='model', hue_order=['RNACNN_pete', 'Mikl all', 'Random baseline'])
#plt.xlim(xmin=0.5)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
tmp

In [ ]:
total[['roc_auc', 'model']].groupby('model').mean()

In [ ]:
total[['roc_auc', 'model']].groupby('model').mean()

In [ ]:
total[['pr_auc', 'model']].groupby('model').mean()

In [ ]:
import collections

In [ ]:
import scipy.stats

In [ ]:
acc = []
for m in sorted(total['model'].unique()):
    acc.append({
        'Method': m,
        'Mean AURoC': total[total['model'] == m]['roc_auc'].mean(),
        'Standard Deviation AURoC': total[total['model'] == m]['roc_auc'].std(),
        'Mean AUPR': total[total['model'] == m]['pr_auc'].mean(),
        'Standard Deviation AUPR': total[total['model'] == m]['pr_auc'].std(),
    })
total_perfs = pd.DataFrame.from_records(acc)
total_perfs.sort_values('Mean AUPR', ascending=False)

In [ ]:
from Bio import SeqIO

In [ ]:
def evaluate_sequence(model: nn.Module, transcript: str, window_size: int, start: int = 0, end: Optional[int] = None):
    acc = []
    if end is None:
        end = len(transcript) - window_size
        
    for i in range(start, end, 3):
        acc.append({
            'sequence': str(transcript[i:i+window_size]),
            'start': i,
            'end': i+window_size,
            'peak': int(i + (window_size/2))
        })
    return acc

In [ ]:
df.iloc[0].sequence[30:50]

In [ ]:
df.iloc[0]

In [ ]:
torch.save(model.state_dict(), 'rnacnn.pth')